# **Step1_AI면접관 Agent v1.0**

## **0. 미션**

### 미션① : 사전 준비

* 1) 면접 사전 준비 절차 구현
    * 이력서/자소서 인식
    * State 정의
    * 서류 분석 : 요약 및 키워드 도출 (LLM)
    * 질문 전략 수립 (LLM)

* 2) 위 항목을 각각 함수나 클래스로 구성한 후, 미션① 을 하나의 함수로 묶기


### 미션② : 면접 Agent

* 1) 면접 절차 구현
    * 답변 입력
    * 답변 평가 : 관점 별, 상, 중, 하 평가 (LLM)
    * 인터뷰 진행 여부 판단 : 추가질문? 인터뷰 종료? (LLM)
    * 질문 생성 : 심화질문 생성 (LLM)
    * 최종 출력 : 질문/답변/평가 결과 출력

* 2) 위 항목을 각각 함수로 구성한 후, 미션 ②를 하나의 Agent로 구성하기


## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

In [2]:
!pip install -r /content/drive/MyDrive/project_genai/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

#### 2) 라이브러리 로딩

In [3]:
import pandas as pd
import numpy as np
import os
import ast
import fitz  # PyMuPDF
from docx import Document
import random
import openai
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langgraph.graph import StateGraph, START, END

### (3) OpenAI API Key 확인

In [4]:
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/project_genai/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [5]:
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-kSJB13pxyxL5xcM6dL--jE


## **2. 미션① : 사전 준비**

### (1) 파일 입력

* Resume 업로드 함수 제공
    * 대상 파일 : word(.docx) pdf 파일로 작성된 이력서/자소서를 입력 받아
    * text를 추출하여 저장




In [56]:
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

### (2) State 선언

* 각 노드의 입출력 관리를 위한 State 구성
    * 각 함수(노드)에서 채워가며 관리해야 할 정보를 도출
    * 이를 하나의 State로 정의
    * 가급적 구조화된 정보로 저장 권장 : str 보다는 List나 Dict
    * 개발하면서 정보가 추가됨


In [80]:
from typing import TypedDict, List, Dict

class InterviewState(TypedDict):
    resume_text: str
    resume_summary: str
    resume_keywords: List[str]
    question_strategy: Dict[str, Dict]

    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation: Dict[str, str]
    next_step: str
    retry_count: int       # 재질문 카운트 (0~3)
    question_index: int    # 현재 질문 번호 (1~3)

In [81]:
# 파일에서 읽어 State 초기화
file_path = path + 'Resume_sample.pdf'

# 텍스트 추출
resume_text = extract_text_from_file(file_path)
resume_text

'<이력서> \n홍길동 (Gil-dong Hong) \n이메일: gildong.hong@example.com \n전화번호: 010-1234-5678 \n학력 \n- 한국대학교 전기정보공학부 학사 (2018.03 ~ 2022.02) \n  GPA: 3.91 / 4.3, 전공과목: 머신러닝, 데이터마이닝, 신호처리 \n경력 \n- KT, AI 연구소 인턴 (2021.07 ~ 2021.12) \n  • OCR 기반 문서 처리 시스템 고도화 \n  • Tesseract + 딥러닝 후처리 파이프라인 설계 \n  • 사내 법률문서 정제 정확도 12% 개선 \n- 빅데이터 학생연합 (BDSA) 기술부장 (2020.03 ~ 2021.02) \n  • Python 기반 크롤러 및 Flask API 개발 \n  • 공공데이터 기반 부동산 가격 예측 프로젝트 리드 \n프로젝트 \n- AI 면접관 시스템 개발 (졸업 과제) \n  • OpenAI GPT + Streamlit + FAISS 기반 질문-응답 시스템 구현 \n  • 이력서 기반 질문 자동 생성 + 답변 피드백 제공 \n- 딥러닝 기반 교통량 예측 (교과목 프로젝트) \n  • LSTM 기반 모델 + 서울시 교통데이터 \n  • MAE 15% 이하로 개선 \n기술 스택 \n- Python, PyTorch, TensorFlow, OpenCV \n- MySQL, MongoDB, Git, Docker \n\n- 영어 (TOEIC 915, 영어면접 가능) \n수상 및 자격 \n- SKT Big Data Challenge 2021 장려상 \n- 정보처리기사 (2022.05 취득) \n기타 \n- Github: github.com/gildong-ai \n- 블로그: blog.naver.com/gildong_dev \n \n \n\n<자기소개서> \n1. 본인 성격의 강/약점에 대해서 실제 사례를 포함하여 작성해 주세요. \n무엇인가 한번 빠져들면 해결하거나 성취할 때까지 모든 열정/노력을 쏟아붓는 성격으\n로, 그 과정에서 큰 어려

In [82]:
# 테스트를 위한 초기 상태 구성
initial_state: InterviewState = {
    "resume_text": resume_text,
    "resume_summary": '',
    "resume_keywords": [],
    "question_strategy": {},

    "current_question": '',
    "current_answer": '',
    "current_strategy": '',
    "conversation": [],
    "evaluation": [],
    "next_step" : ''
}

initial_state

{'resume_text': '<이력서> \n홍길동 (Gil-dong Hong) \n이메일: gildong.hong@example.com \n전화번호: 010-1234-5678 \n학력 \n- 한국대학교 전기정보공학부 학사 (2018.03 ~ 2022.02) \n  GPA: 3.91 / 4.3, 전공과목: 머신러닝, 데이터마이닝, 신호처리 \n경력 \n- KT, AI 연구소 인턴 (2021.07 ~ 2021.12) \n  • OCR 기반 문서 처리 시스템 고도화 \n  • Tesseract + 딥러닝 후처리 파이프라인 설계 \n  • 사내 법률문서 정제 정확도 12% 개선 \n- 빅데이터 학생연합 (BDSA) 기술부장 (2020.03 ~ 2021.02) \n  • Python 기반 크롤러 및 Flask API 개발 \n  • 공공데이터 기반 부동산 가격 예측 프로젝트 리드 \n프로젝트 \n- AI 면접관 시스템 개발 (졸업 과제) \n  • OpenAI GPT + Streamlit + FAISS 기반 질문-응답 시스템 구현 \n  • 이력서 기반 질문 자동 생성 + 답변 피드백 제공 \n- 딥러닝 기반 교통량 예측 (교과목 프로젝트) \n  • LSTM 기반 모델 + 서울시 교통데이터 \n  • MAE 15% 이하로 개선 \n기술 스택 \n- Python, PyTorch, TensorFlow, OpenCV \n- MySQL, MongoDB, Git, Docker \n\n- 영어 (TOEIC 915, 영어면접 가능) \n수상 및 자격 \n- SKT Big Data Challenge 2021 장려상 \n- 정보처리기사 (2022.05 취득) \n기타 \n- Github: github.com/gildong-ai \n- 블로그: blog.naver.com/gildong_dev \n \n \n\n<자기소개서> \n1. 본인 성격의 강/약점에 대해서 실제 사례를 포함하여 작성해 주세요. \n무엇인가 한번 빠져들면 해결하거나 성취할 때까지 모든 열정/노력을 쏟아붓는 성격으

### (3) Resume 분석

* 목적
    * 개인화된 질문을 뽑기 위한 이력서 핵심 내용 및 키워드 도출
    * 질문 전략을 수립하기 위한 기본 자료로 사용
* 입출력
    * 입력 : 이력서/자소서 text
    * 출력 : 핵심 요약 str, 주요 키워드 List
* 처리
    * 프롬프트 템플릿 구성하여 LLM 활용한 처리
    * 요약과 키워드 도출을 한꺼번에 할 수도 있고, 단계를 나눠서 요약 따로, 키워드 도출 따로 진행도 가능


In [83]:
def analyze_resume(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    prompt = PromptTemplate(
        template='''
        당신은 면접을 미리 준비하는 면접관입니다.
        당신은 다음 자기소개서를 보고 5줄로 요약해서 미리 준비하고자 합니다.
        결과는 자기소개서 확인 후 5줄 내외로 요약합니다.
        ---
        자기소개서 : {resume_text}
        ---
        ''',
        input_variables=["resume_text"]
    )
    resume_summary = (prompt | llm).invoke(state["resume_text"]).content

    prompt = PromptTemplate(
        template='''
        당신은 면접을 미리 준비하는 면접관입니다.
        다음 자기소개서를 보고 중요하다고 판단되는 키워드 10개 가량 생각하려고 합니다.
        결과는 각 키워드를 , 로 구분하여 나열합니다.
        답변 예시 : [... , ... , ...]
        ---
        자기소개서 : {resume_text}
        ---
        ''',
        input_variables=["resume_text"]
    )
    resume_keywords = (prompt | llm | CommaSeparatedListOutputParser()).invoke(state["resume_text"])


    # return 코드는 제공합니다.
    return {
        **state,
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
    }


In [84]:
state = initial_state
state = analyze_resume(state)
print(type(state["resume_summary"]))
print(state["resume_summary"][:100]," ...")
print('-'*20)
print(type(state["resume_keywords"]))
print(state["resume_keywords"][:100]," ...")

<class 'str'>
홍길동은 전기정보공학을 전공하며 머신러닝과 데이터마이닝에 대한 깊은 이해를 갖춘 인재입니다. KT AI 연구소에서 인턴으로 OCR 시스템을 개선한 경험이 있으며, 빅데이터 학생연합  ...
--------------------
<class 'list'>
['[홍길동', '전기정보공학', '머신러닝', 'AI 연구소 인턴', 'Python', '딥러닝', '프로젝트 경험', '알고리즘', '서버 개발', '코드 리뷰]']  ...


In [85]:
def generate_question_strategy(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

    prompt = PromptTemplate(
        template='''
        당신은 면접 준비를 하는 면접관입니다.
        당신은 요약된 자기소개서와 질문 키워드를 받아 어떤 질문을 해야할지 질문 전략을 생각하려 합니다.
        질문 전략의 분야는 다음과 같습니다. : ["경력 및 경험", "동기 및 커뮤니케이션","논리적 사고"]
        답변은 아래와 같은 구조를 따릅니다.
        {{
            "경력 및 경험" : {{
                "질문 방향성" : "...",
                "예시 질문" : "..."
             }},
             "동기 및 커뮤니케이션" : {{
                "질문 방향성" : "...",
                "예시 질문" : "..."
             }},
             "논리적 사고":{{
                "질문 방향성" : "...",
                "예시 질문" : "..."
             }}
        }}

        ---
        요약된 자기 소개서 : {resume_summary}
        ---
        질문 키워드 : {resume_keywords}
        ---
        ''',
        input_variables=["resume_summary", "resume_keywords"]
    )
    resume_summary = state['resume_summary']
    resume_keywords = state['resume_keywords']

    response = (prompt | llm | StrOutputParser()).invoke(
        {
            "resume_summary": resume_summary,
            "resume_keywords": resume_keywords
        }
    )
    import json
    strategy_dict = json.loads(response)

    # return 코드는 제공합니다.
    return {
        **state,
        "question_strategy": strategy_dict
    }


In [86]:
state = generate_question_strategy(state)
print(type(state['question_strategy']))
print(state['question_strategy'])

<class 'dict'>
{'경력 및 경험': {'질문 방향성': '홍길동의 인턴 경험과 프로젝트 리더십을 통해 얻은 구체적인 기술적 성과와 배운 점을 파악하고자 합니다.', '예시 질문': 'KT AI 연구소에서 인턴으로 OCR 시스템을 개선한 경험에 대해 구체적으로 설명해 주실 수 있나요? 어떤 기술을 사용했고, 어떤 결과를 얻었는지 궁금합니다.'}, '동기 및 커뮤니케이션': {'질문 방향성': 'KK기술에 지원한 동기와 팀 내에서의 커뮤니케이션 방식에 대해 알아보려 합니다.', '예시 질문': 'KK기술에서 안정적인 환경에서 실무 경험을 쌓고 싶다고 하셨는데, 구체적으로 어떤 경험을 통해 성장하고 싶으신가요? 또한, 팀원들과의 협업에서 중요하게 생각하는 점은 무엇인가요?'}, '논리적 사고': {'질문 방향성': '문제 해결에 대한 열정과 알고리즘 문제 해결 능력을 평가하고자 합니다.', '예시 질문': '알고리즘 문제 해결에 꾸준히 노력하고 있다고 하셨는데, 최근에 해결한 알고리즘 문제 중 가장 도전적이었던 것은 무엇이었고, 어떻게 접근하셨나요?'}}


### (5) 하나로 묶기

* 목적
    * 사전 준비 절차는 한꺼번에 1회성으로 실행되도록 각 단계를 하나의 함수로 묶음
* 입출력
    * 입력 : 이력서/자소서 파일
    * 출력 : State
* 처리
    * 함수 이름 : preProcessing_Interview
    * 구축한 함수들을 순차적으로 실행되도록 구성
    * 첫번째 질문 생성
        * 질문 전략 수립 시 생성한 ‘경력 및 경험’의
        * 예시 질문을 첫번째 질문으로 구성하기


In [115]:
def preProcessing_Interview(file_path: str) -> InterviewState:
    # 여기에 코드를 완성합니다.


    # 텍스트 추출
    resume_text = extract_text_from_file(file_path)

    # 초기 상태 구성
    initial_state: InterviewState = {
        "resume_text": resume_text,
        "resume_summary": '',
        "resume_keywords": [],
        "question_strategy": {},

        "current_question": '',
        "current_answer": '',
        "current_strategy": '',
        "conversation": [],
        "evaluation": [],
        "next_step" : '',
        "retry_count": 0,
        "question_index": 1
    }
    state = initial_state
    # 요약추가 특징 추가
    state = analyze_resume(initial_state)
    # 면접질문 추출
    state = generate_question_strategy(state)
    # 첫번째 질문 세팅
    selected_question = state['question_strategy']['경력 및 경험']['예시 질문']
    # return 코드는 제공합니다.
    return {
            **state,
            "current_question": selected_question,
            "current_strategy": "경력 및 경험"
            }

In [116]:
# 파일에서 읽어 State 초기화
file_path = path + 'Resume_sample.pdf'

state = preProcessing_Interview(file_path)
state

{'resume_text': '<이력서> \n홍길동 (Gil-dong Hong) \n이메일: gildong.hong@example.com \n전화번호: 010-1234-5678 \n학력 \n- 한국대학교 전기정보공학부 학사 (2018.03 ~ 2022.02) \n  GPA: 3.91 / 4.3, 전공과목: 머신러닝, 데이터마이닝, 신호처리 \n경력 \n- KT, AI 연구소 인턴 (2021.07 ~ 2021.12) \n  • OCR 기반 문서 처리 시스템 고도화 \n  • Tesseract + 딥러닝 후처리 파이프라인 설계 \n  • 사내 법률문서 정제 정확도 12% 개선 \n- 빅데이터 학생연합 (BDSA) 기술부장 (2020.03 ~ 2021.02) \n  • Python 기반 크롤러 및 Flask API 개발 \n  • 공공데이터 기반 부동산 가격 예측 프로젝트 리드 \n프로젝트 \n- AI 면접관 시스템 개발 (졸업 과제) \n  • OpenAI GPT + Streamlit + FAISS 기반 질문-응답 시스템 구현 \n  • 이력서 기반 질문 자동 생성 + 답변 피드백 제공 \n- 딥러닝 기반 교통량 예측 (교과목 프로젝트) \n  • LSTM 기반 모델 + 서울시 교통데이터 \n  • MAE 15% 이하로 개선 \n기술 스택 \n- Python, PyTorch, TensorFlow, OpenCV \n- MySQL, MongoDB, Git, Docker \n\n- 영어 (TOEIC 915, 영어면접 가능) \n수상 및 자격 \n- SKT Big Data Challenge 2021 장려상 \n- 정보처리기사 (2022.05 취득) \n기타 \n- Github: github.com/gildong-ai \n- 블로그: blog.naver.com/gildong_dev \n \n \n\n<자기소개서> \n1. 본인 성격의 강/약점에 대해서 실제 사례를 포함하여 작성해 주세요. \n무엇인가 한번 빠져들면 해결하거나 성취할 때까지 모든 열정/노력을 쏟아붓는 성격으

## **3. 미션② : 면접 Agent**

### (1) 답변 입력

* 사용자 답변을 입력 받아 State의 current_answer에 저장

In [117]:
def update_current_answer(state: InterviewState, user_answer: str) -> InterviewState:
    return {
        **state,
        "current_answer": user_answer.strip()
    }


In [118]:
answer = """
KT AI 연구소에서 OCR 시스템을 개선할 당시, 가장 큰 문제는 이미지 품질이 낮거나 배경이 복잡한 문서에서 문자 인식 정확도가 크게 떨어진다는 것이었습니다. 특히 모바일 스캔 이미지에서 노이즈나 왜곡이 많아, 기존 OCR 모델로는 중요한 정보를 누락하거나 잘못 인식하는 경우가 많았습니다.

이를 해결하기 위해 먼저 데이터셋을 분석하고, 문제가 되는 이미지 유형에 특화된 전처리 알고리즘을 도입했습니다. 예를 들어, Adaptive Thresholding과 Geometric Correction을 통해 이미지 품질을 개선했고, CRNN 기반 OCR 모델을 개선하여 비정형 문서에도 더 강인하게 작동하도록 조정했습니다.

그 결과, 문서 전체 인식 정확도가 약 12% 향상되었고, 특히 고객이 실제로 사용하는 모바일 앱 내 OCR 정확도는 85%에서 95% 수준까지 올라갔습니다. 이 개선안은 이후 사내 다른 프로젝트에도 확장 적용되었습니다.
"""

state = update_current_answer(state, answer)
print(state['current_answer'])

KT AI 연구소에서 OCR 시스템을 개선할 당시, 가장 큰 문제는 이미지 품질이 낮거나 배경이 복잡한 문서에서 문자 인식 정확도가 크게 떨어진다는 것이었습니다. 특히 모바일 스캔 이미지에서 노이즈나 왜곡이 많아, 기존 OCR 모델로는 중요한 정보를 누락하거나 잘못 인식하는 경우가 많았습니다.

이를 해결하기 위해 먼저 데이터셋을 분석하고, 문제가 되는 이미지 유형에 특화된 전처리 알고리즘을 도입했습니다. 예를 들어, Adaptive Thresholding과 Geometric Correction을 통해 이미지 품질을 개선했고, CRNN 기반 OCR 모델을 개선하여 비정형 문서에도 더 강인하게 작동하도록 조정했습니다.

그 결과, 문서 전체 인식 정확도가 약 12% 향상되었고, 특히 고객이 실제로 사용하는 모바일 앱 내 OCR 정확도는 85%에서 95% 수준까지 올라갔습니다. 이 개선안은 이후 사내 다른 프로젝트에도 확장 적용되었습니다.


### (2) 답변 평가
* 목적 : 면접자의 답변이 적절한지 평가
* 입출력
    * 입력 : State
    * 출력 : State(답변, 평가가 포함된)
* 평가
    * 평가 항목 : 두 가지(질문과의 연관성, 답변의 구체성)로 정하되 추가 가능
    * 평가 등급 : 상, 중, 하(상, 중, 하에 대한 등급 기준 설명 예시)

* 질문 답변 내용 conversation에 추가



In [149]:
def evaluate_answer(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

    prompt = PromptTemplate(
        template = '''
        당신은 인공지능 면접관입니다.
        다음 정보를 보고 면접자의 질문이 적절한지 판단합니다.

        당신은 내용을 보고 '질문의 연관성'과 '답변의 구체성'을  바탕으로 평가 등급을 4가지 분류로 나뉩니다.
        그후 평가 이유 및 보완점을 서술합니다.
        보완점을 구체적으로 서술한다.

        질문의 연관성 평가 기준

        1.매우 우수
        질문의 의도를 완벽히 이해하고 답변하며, 직무·회사·상황 맥락까지 자연스럽게 연결한다. 질문의 핵심뿐 아니라 확장된 의미까지 파악하여 심화된 내용을 말할 수 있다.

        2.우수
        질문의 의도를 잘 이해했고, 핵심 내용을 빠짐없이 언급한다. 다만 추가적인 배경이나 심화 내용은 다소 부족할 수 있다.

        3.보통
        질문과 부분적으로만 연결되며, 핵심 내용 중 일부가 빠지거나 표면적인 이해에 머무른다. 약간 질문에서 벗어난 부분이 느껴질 수 있다.

        4.미흡
        질문과 거의 관련 없거나 질문 의도를 잘못 이해했다. 답변이 엉뚱한 방향으로 흘러가거나 주제를 벗어난다. 대답이 짧거나 질문에 전혀 대답하지 않거나 ‘모르겠습니다’, ‘열심히 하겠습니다’ 같은 형식적 표현에 그친다.

        답변의 구체성 평가 기준

        1.매우 우수
        구체적 경험, 데이터, 행동, 결과, 배운 점까지 매우 상세히 언급한다.

        2.우수
        구체적 사례, 경험, 행동, 결과 중 2~3가지를 포함한다. 답변이 비교적 명확하고 구체적이지만, 일부 세부 정보는 부족할 수 있다.

       3. 보통
        일부 구체적인 내용은 있으나 예시나 경험이 부족하거나, 경험은 언급했으나 결과나 배운 점이 빠졌다. 다소 추상적인 답변이 섞여 있다.

        4.미흡
        매우 추상적이고 원론적인 답변으로, 개인 경험이나 구체적 사례가 거의 없다. 대답이 짧거나 질문에 전혀 대답하지 않거나 ‘모르겠습니다’, ‘열심히 하겠습니다’,“최선을 다하겠습니다” 같은 형식적 표현에 그친다.

        ---
        이력서 요약 : {resume_summary}

        이력서 키워드 : {resume_keywords}

        면접 질문 전략 : {current_strategy}

        면접관의 질문 : {current_question}

        면접자의 답변 : {current_answer}

        ---
        주의: 반드시 아래 형식으로 JSON 하나만 출력하세요. 예:
        {{
        "질문의 연관성": "매우 우수",
        "답변의 구체성": "미흡"
        "이유 및 보완점": "답변이 질문 의도에 부합하지만 구체적 사례가 부족합니다."
        }}
        그 외 다른 문장은 절대 출력하지 마세요.
        특히, '안녕하세요', '파이팅', '잘 부탁드립니다', '모르겠습니다' 같은 인사말이나 형식적, 질문과 연관 없는 답변은 반드시 '질문의 연관성: 미흡', '답변의 구체성: 미흡'으로 평가하고, 그 이유를 명시하세요.

        ''',
        input_variables = ["resume_summary", "resume_keywords", "current_strategy", "current_question", "current_answer"]
    )
    response = (prompt | llm | StrOutputParser()).invoke({
        "resume_summary": state["resume_summary"],
        "resume_keywords": state["resume_keywords"],
        "current_strategy": state["current_strategy"],
        "current_question": state["current_question"],
        "current_answer": state["current_answer"]
    })

    import json
    evaluation = json.loads(response)

    conversation = state['conversation']
    conversation.append({
        "question": state["current_question"],
        "answer": state["current_answer"],
        "evaluation": evaluation
    })
    state["conversation"] = conversation

    # return 코드는 제공합니다.
    return {
        **state,
        "evaluation": evaluation
    }


In [120]:
# state = evaluate_answer(state)
# print(type(state["evaluation"]))
# print(state["evaluation"])
# print('-'*50)
# print(type(state["conversation"]))
# print(state["conversation"])

### (3) 인터뷰 진행 검토

* 목적
    * 인터뷰가 길어지지 않도록 진행 제한
        * 질문&답변 이 3회 초과 되었으면 멈춤 (end)
        * 아니면 추가 질문 생성 (additional_question)
* 입출력
    * 입력 : State
    * 출력 : State(next_step에 내용이 포함된)
* LLM 없이 규칙 기반(if ~ else)로만 구현


In [133]:
MAX_RETRY_COUNT = 3  # 재질문 최대 횟수

def decide_next_step(state: InterviewState) -> InterviewState:
    eval_rel = state["evaluation"].get("질문의 연관성", "")
    eval_detail = state["evaluation"].get("답변의 구체성", "")

    retry_count = state.get("retry_count", 0)
    question_index = state.get("question_index", 1)

    # 미흡 or 보통 → 재질문 카운트 증가
    if (eval_rel in ["미흡", "보통"]) or (eval_detail in ["미흡", "보통"]):
        retry_count += 1
        if retry_count > MAX_RETRY_COUNT:
            retry_count = 0
            question_index += 1
    else:
        retry_count = 0
        question_index += 1

    if question_index > 3:
        next_step = "end"
    else:
        next_step = "additional_question"

    return {
        **state,
        "next_step": next_step,
        "retry_count": retry_count,
        "question_index": question_index
    }


In [134]:
# state = decide_next_step(state)
# print(state['next_step'])

### (4) 질문 생성

* 목적 : 이전 질문 답변을 검토한 후 추가 심화 질문 생성
* 입출력
    * 입력 : State
    * 출력 : State(current_question 에 질문 추가, current_answer 는 빈 값)
* 처리
    * 이력서 요약, 키워드, 질문전략, 이전 질문과 답변, 평가를 기반으로
    * 지원자의 사고력, 문제 해결 방식, 혹은 기술적 깊이를 더 확인할 수 있는 심화 인터뷰 질문 생성


In [142]:
def generate_question(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o-mini")
    retry_count = state.get("retry_count", 0)
    question_index = state.get("question_index", 1)

    # 전략 순서 고정
    strategy_list = ["경력 및 경험", "동기 및 커뮤니케이션", "논리적 사고"]
    current_idx = min(question_index - 1, len(strategy_list) -1)
    current_strategy = strategy_list[current_idx]

    prompt = PromptTemplate(
        template='''
        당신은 인공지능 면접관입니다.
        다음 정보를 보고 면접자에게 질문할 심화 질문을 1개 생성합니다.
        ---
        이력서 요약 : {resume_summary}
        이력서 키워드 : {resume_keywords}
        면접 질문 전략 : {current_strategy}
        면접관의 질문 : {current_question}
        면접자의 대답 : {current_answer}
        면접관의 평가 : {evaluation}
        ''',
        input_variables=["resume_summary", "resume_keywords", "current_strategy", "current_question", "current_answer", "evaluation"]
    )
    response = (prompt | llm | StrOutputParser()).invoke({
        "resume_summary": state["resume_summary"],
        "resume_keywords": state["resume_keywords"],
        "current_strategy": current_strategy,
        "current_question": state["current_question"],
        "current_answer": state["current_answer"],
        "evaluation": state["evaluation"]
    })

    # 질문 이름 붙이기
    if retry_count > 0:
        question_title = f"재질문 {question_index}-{retry_count}"
    else:
        question_title = f"질문 {question_index}"

    return {
        **state,
        "current_question": f"{question_title}: {response.strip()}",
        "current_answer": "",
        "current_strategy": current_strategy
    }


In [143]:
state = generate_question(state)
print(state['current_question'])

재질문 2-2: "앞서 말씀하신 팀 프로젝트에서의 역할과 사용한 기술에 대해 좀 더 구체적으로 설명해 주실 수 있나요? 특히, 프로젝트의 목표를 달성하기 위해 어떤 특정한 머신러닝 기법이나 데이터 처리 방법을 적용했는지, 그리고 그 과정에서 팀원들과의 소통을 위해 어떤 도구를 사용했는지에 대한 구체적인 사례를 들어 주시면 좋겠습니다. 또한, 프로젝트 진행 중 발생한 갈등 상황이 있었다면, 그 갈등을 어떻게 해결했는지에 대해서도 말씀해 주시면 감사하겠습니다."


### (5) 인터뷰 피드백 보고서
* 목적 : 인터뷰를 종료하며, 인터뷰 내용과 평가내용 화면 출력
* 입출력
    * 입력 : State
    * 출력 : State
* 처리 : 질문 / 답변 / 평가에 대한 순차적 출력


In [144]:
# state['conversation'][0]['evaluation']

In [145]:
def summarize_interview(state: InterviewState) -> InterviewState:
    print("-" * 50)
    print("[면접 요약]")
    print("-" * 50)
    print(f"[이력서/자기소개서 요약] {state['resume_summary']}")
    print("-" * 50)

    for i, turn in enumerate(state["conversation"]):
        print(f"[질문 {i+1}] {turn['question']}")
        print(f"[답변 {i+1}] {turn['answer']}")
        eval_result = state["conversation"][i]['evaluation']
        print(f"[평가] 질문의 연관성: {eval_result['질문의 연관성']}, 답변의 구체성: {eval_result['답변의 구체성']}")
        if "이유 및 보완점" in eval_result:
            print(f"[이유 및 보완점] {eval_result['이유 및 보완점']}")
        print("-" * 50)

    return state

### (6) 면접 Agent

* 목적 : [사용자 답변 입력 ~ 질문 출력/종료] 까지 한 Agent로 구성
* Agent 입출력
    * 입력 : State, 면접자 답변
    * 출력 : State, 인터뷰 보고서
* 처리 : 각 함수(노드)를 흐름에 맞게 연결하기


In [150]:
# 분기 판단 함수
def route_next(state: InterviewState) -> Literal["generate_question", "summarize_interview"]:
    return "summarize_interview" if state["next_step"] == "end" else "generate_question"

# 그래프 정의 시작
builder = StateGraph(InterviewState)

# 노드 추가
builder.add_node(evaluate_answer, "evaluate_answer")
builder.add_node(decide_next_step, "decide_next_step")
builder.add_node(generate_question, "generate_question")
builder.add_node(summarize_interview, "summarize_interview")

# 노드 연결
builder.add_edge(START, "evaluate_answer")
builder.add_edge("evaluate_answer", "decide_next_step")
builder.add_conditional_edges("decide_next_step", route_next, {
    "summarize_interview": "summarize_interview",
    "generate_question": "generate_question"
})

builder.add_edge("generate_question", END)
builder.add_edge("summarize_interview", END)

# 컴파일
graph = builder.compile()


## **4. 시스템 실행**

미션1,2에서 수행한 결과를 통합 테스트 해 봅시다.

### (1) 인터뷰 사전준비 작업

In [151]:
# 파일 입력
file_path = path + 'Resume_sample.pdf'
state = preProcessing_Interview(file_path)
state

{'resume_text': '<이력서> \n홍길동 (Gil-dong Hong) \n이메일: gildong.hong@example.com \n전화번호: 010-1234-5678 \n학력 \n- 한국대학교 전기정보공학부 학사 (2018.03 ~ 2022.02) \n  GPA: 3.91 / 4.3, 전공과목: 머신러닝, 데이터마이닝, 신호처리 \n경력 \n- KT, AI 연구소 인턴 (2021.07 ~ 2021.12) \n  • OCR 기반 문서 처리 시스템 고도화 \n  • Tesseract + 딥러닝 후처리 파이프라인 설계 \n  • 사내 법률문서 정제 정확도 12% 개선 \n- 빅데이터 학생연합 (BDSA) 기술부장 (2020.03 ~ 2021.02) \n  • Python 기반 크롤러 및 Flask API 개발 \n  • 공공데이터 기반 부동산 가격 예측 프로젝트 리드 \n프로젝트 \n- AI 면접관 시스템 개발 (졸업 과제) \n  • OpenAI GPT + Streamlit + FAISS 기반 질문-응답 시스템 구현 \n  • 이력서 기반 질문 자동 생성 + 답변 피드백 제공 \n- 딥러닝 기반 교통량 예측 (교과목 프로젝트) \n  • LSTM 기반 모델 + 서울시 교통데이터 \n  • MAE 15% 이하로 개선 \n기술 스택 \n- Python, PyTorch, TensorFlow, OpenCV \n- MySQL, MongoDB, Git, Docker \n\n- 영어 (TOEIC 915, 영어면접 가능) \n수상 및 자격 \n- SKT Big Data Challenge 2021 장려상 \n- 정보처리기사 (2022.05 취득) \n기타 \n- Github: github.com/gildong-ai \n- 블로그: blog.naver.com/gildong_dev \n \n \n\n<자기소개서> \n1. 본인 성격의 강/약점에 대해서 실제 사례를 포함하여 작성해 주세요. \n무엇인가 한번 빠져들면 해결하거나 성취할 때까지 모든 열정/노력을 쏟아붓는 성격으

### (2) Agent 실행

In [152]:
# 사용자 응답 루프
while True:
    print("\n[질문]")
    print(state["current_question"])

    state["current_answer"] = input("\n[답변 입력]:\n")

    # 그래프 실행: 평가 → 판단 → 다음 질문 생성 or 종료
    state = graph.invoke(state)

    if state["next_step"] == "end":
        break



[질문]
KT AI 연구소에서 인턴으로 OCR 시스템을 개선한 구체적인 과정과 그 결과에 대해 설명해 주실 수 있나요? 또한, 빅데이터 학생연합에서 프로젝트를 리드하면서 어떤 도전과제를 겪었고, 이를 어떻게 해결했는지 말씀해 주세요.

[답변 입력]:
팀원들과 잘 협력해서 성공했습니다

[질문]
재질문 1-1: 면접관의 심화 질문: "OCR 시스템 개선 과정에서 구체적으로 어떤 기술적 접근 방식을 사용하였고, 그 과정에서 발생한 특정 문제는 무엇이었으며, 이를 해결하기 위해 어떤 방법을 적용했는지 자세히 설명해 주실 수 있나요? 또한, 빅데이터 학생연합에서 리드한 프로젝트의 목표와 그 과정에서 팀원들과의 협업 방식에 대해 구체적인 사례를 들어 말씀해 주세요."

[답변 입력]:
“OCR 시스템 개선 프로젝트에서 팀원들과 정기적으로 회의를 통해 진행 상황을 공유했고, 각자 맡은 파트를 열심히 수행했습니다. 의견 충돌이 있었던 적도 있었지만, 결국에는 팀장님의 중재로 잘 해결되었습니다. 이런 경험 덕분에 협업의 중요성을 배우게 되었고, KK기술에서도 팀원들과 잘 협력할 수 있을 거라 생각합니다.”

[질문]
재질문 1-2: 면접관의 심화 질문: "OCR 시스템 개선 과정에서 사용한 특정 알고리즘이나 도구에 대해 말씀해 주실 수 있나요? 그리고 그 과정에서 직면했던 데이터 품질 문제나 성능 저하와 같은 구체적인 문제를 어떻게 진단하고 해결했는지, 그 과정에서 어떤 데이터를 활용했는지에 대해 자세히 설명해 주세요. 또한, 빅데이터 학생연합에서 리드한 프로젝트의 목표가 무엇이었고, 팀원들과의 협업 과정에서 발생한 갈등을 어떻게 해결했는지 구체적인 사례를 들어 말씀해 주시면 좋겠습니다."

[답변 입력]:
OCR 결과에 딥러닝 기반 후처리 파이프라인을 설계하여, 잘못 인식된 부분을 교정하고 문서 구조에 맞게 정제하는 작업을 추가했습니다. 이 과정을 통해 사내 법률문서 정제 정확도를 12% 개선할 수 있었고, 기술적으로도 매우 값진 경험이었습니다.

[질문]
질문 2: 